In [1]:
#import libraries
import os
import glob
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
import regionmask
import statsmodels.api as sm




In [2]:
# path to ERA5-Land hourly data
ERA5_DIR = "/Users/niko2485/Library/CloudStorage/OneDrive-UCB-O365/Desktop/EarthScienceCourse/Final project/era5land_hourly"

# defining path to output directory
OUT_DIR = "/Users/niko2485/Library/CloudStorage/OneDrive-UCB-O365/Desktop/EarthScienceCourse/Final project/output"
os.makedirs(OUT_DIR, exist_ok=True)

# path to the shapefile
SHAPEFILE = "/Users/niko2485/Library/CloudStorage/OneDrive-UCB-O365/Desktop/EarthScienceCourse/Final project/IND/AgroMaps/Asia/shapefiles/IND/admin1/ind.shp"  

print("SHAPEFILE =", SHAPEFILE)
print("Exists?   =", os.path.exists(SHAPEFILE))


SHAPEFILE = /Users/niko2485/Library/CloudStorage/OneDrive-UCB-O365/Desktop/EarthScienceCourse/Final project/IND/AgroMaps/Asia/shapefiles/IND/admin1/ind.shp
Exists?   = True


In [ ]:
# list all ERA5-Land files
real_files = sorted(glob.glob(os.path.join(ERA5_DIR, "*_real.nc")))
print("Total files:", len(real_files))
real_files[:5]


Total files: 44


['/Users/niko2485/Library/CloudStorage/OneDrive-UCB-O365/Desktop/EarthScienceCourse/Final project/era5land_hourly/era5land_t2m_india_MAMJ_1980_real.nc',
 '/Users/niko2485/Library/CloudStorage/OneDrive-UCB-O365/Desktop/EarthScienceCourse/Final project/era5land_hourly/era5land_t2m_india_MAMJ_1981_real.nc',
 '/Users/niko2485/Library/CloudStorage/OneDrive-UCB-O365/Desktop/EarthScienceCourse/Final project/era5land_hourly/era5land_t2m_india_MAMJ_1982_real.nc',
 '/Users/niko2485/Library/CloudStorage/OneDrive-UCB-O365/Desktop/EarthScienceCourse/Final project/era5land_hourly/era5land_t2m_india_MAMJ_1983_real.nc',
 '/Users/niko2485/Library/CloudStorage/OneDrive-UCB-O365/Desktop/EarthScienceCourse/Final project/era5land_hourly/era5land_t2m_india_MAMJ_1984_real.nc']

In [4]:
# create a dictionary mapping year to file path
file_by_year = {}

for f in real_files:
    base = os.path.basename(f)
    year = int(base.split("_")[-2])     # <-- FIXED
    file_by_year[year] = f

years_to_use = sorted(file_by_year.keys())
years_to_use[:10]


[1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989]

In [5]:
# use all years:
years_to_use = sorted(file_by_year.keys())
print("Years to use:", years_to_use)  


Years to use: [1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]


In [6]:
# read the shapefile using geopandas
zones = gpd.read_file(SHAPEFILE)

# use the state name column
zone_col = "NAME1_"

if zones.crs is None:
    zones = zones.set_crs("EPSG:4326")
elif zones.crs.to_epsg() != 4326:
    zones = zones.to_crs("EPSG:4326")

regions = regionmask.from_geopandas(zones, names=zone_col)
regions

# create a mapping from zone ID to zone name
zone_id_to_name = zones[zone_col].to_dict()


In [7]:

# open one year just to get the grid
first_year = years_to_use[0]
ds0 = xr.open_dataset(file_by_year[first_year])

print(ds0.coords)

# rename coords so they are 'time', 'lat', 'lon'
ds0 = ds0.rename({
    "valid_time": "time",
    "latitude": "lat",
    "longitude": "lon"
})

# now build static mask (2D: lat x lon)
mask_grid = regions.mask(ds0)  # regionmask now sees lat/lon
mask_grid


Coordinates:
  * valid_time  (valid_time) datetime64[ns] 23kB 1980-03-01 ... 1980-06-30T23...
  * latitude    (latitude) float64 3kB 38.0 37.9 37.8 37.7 ... 6.3 6.2 6.1 6.0
  * longitude   (longitude) float64 2kB 68.0 68.1 68.2 68.3 ... 97.8 97.9 98.0
    number      int64 8B ...
    expver      (valid_time) <U4 47kB ...


<xarray.DataArray 'mask' (lat: 321, lon: 301)> Size: 773kB
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], shape=(321, 301))
Coordinates:
  * lat      (lat) float64 3kB 38.0 37.9 37.8 37.7 37.6 ... 6.4 6.3 6.2 6.1 6.0
  * lon      (lon) float64 2kB 68.0 68.1 68.2 68.3 68.4 ... 97.7 97.8 97.9 98.0
    number   int64 8B 0
Attributes:
    standard_name:  region
    flag_values:    [ 0  1  2  3  4  5  6  7  8  9 10 12 13 14 15 16 17 18 19...
    flag_meanings:  r0 r1 r2 r3 r4 r5 r6 r7 r8 r9 r10 r12 r13 r14 r15 r16 r17...

### Calculating State-Level Heat Metrics from ERA5-Land (1980–2023)

This section of the code loops through every year of ERA5-Land temperature data and computes a set of heat indicators for each Indian state during the pre-monsoon season (March–June). For each year, the script:

1. Opens the hourly temperature dataset and converts values from Kelvin to °C.  
2. Aggregates hourly temperatures into daily maximum temperature (Tmax).  
3. Selects only the months March–June, when India typically experiences its most intense heat.  
4. Applies a state-level spatial mask (from a shapefile) to compute temperature statistics for each region.  
5. For every state, calculates four heat metrics:  
   - **Mean Tmax** for the season  
   - **95th percentile Tmax** (extreme heat)  
   - **Number of days with Tmax ≥ 38°C** (a commonly used threshold for dangerous heat exposure)  
   - **The first day of the year when Tmax ≥ 38°C** (onset of extreme heat)  

Each state-year combination is stored as a row in a list that is later converted into a pandas DataFrame.  
This creates a clean, analysis-ready dataset that summarizes more than 40 years of pre-monsoon heat patterns across India.



In [8]:
# set threshold for hot days
threshold = 38.0  # °C

rows = []

# see how many regions in the mask? 
n_regions = int(np.nanmax(mask_grid.values)) + 1
print("Number of regions in mask:", n_regions)

for year in years_to_use:
    print("Processing year:", year)
    f = file_by_year[year]
    ds_y = xr.open_dataset(f)

    # 1. Rename coords
    ds_y = ds_y.rename({
        "valid_time": "time",
        "latitude": "lat",
        "longitude": "lon"
    })

    # 2. Kelvin → °C conversion
    ds_y["t2m"] = ds_y["t2m"] - 273.15
    ds_y["t2m"].attrs["units"] = "degC"

    # 3. Add time-based coordinates
    t = ds_y["time"]
    ds_y = ds_y.assign_coords(
        date=("time", t.dt.floor("D").data),
        year=("time", t.dt.year.data),
        month=("time", t.dt.month.data),
        doy=("time", t.dt.dayofyear.data),
    )

    # 4. Daily Tmax
    daily = ds_y["t2m"].groupby("date").max("time")
    daily = daily.assign_coords(
        year=("date", daily["date"].dt.year.data),
        month=("date", daily["date"].dt.month.data),
        doy=("date", daily["date"].dt.dayofyear.data),
    )

    # 5. Restrict to March–June
    daily_pre = daily.where(daily["month"].isin([3, 4, 5, 6]), drop=True)

    # 6. Loop over regions
    for rid in range(n_regions):
        # Boolean mask for this region ID
        region_mask = (mask_grid == rid)  # dims: (lat, lon)

        # Skip if this region has no grid cells
        if not bool(region_mask.any()):
            continue

        # Mask out everything except this region
        # Broadcasts region_mask over the date dimension
        tmp = daily_pre.where(region_mask)

        # Average over space → one time series per region
        # dims now: date
        reg_daily = tmp.mean(dim=("lat", "lon"), skipna=True)

        # If we somehow got all NaNs, skip
        if reg_daily.isnull().all():
            continue

        # Coords along 'date'
        doy = reg_daily["doy"]

        # --- Metrics ---

        # 1) Mean Tmax over pre-monsoon days
        mean_tmax = float(reg_daily.mean(dim="date").values)

        # 2) 95th percentile Tmax
        p95_tmax = float(reg_daily.quantile(0.95, dim="date").values)

        # 3) Number of hot days (Tmax ≥ threshold)
        hot_days = int((reg_daily >= threshold).sum(dim="date").values)

        # 4) Onset doy: first day-of-year with Tmax ≥ threshold
        hot_flags = (reg_daily >= threshold)
        doy_hot = doy.where(hot_flags)
        # might be all-NaN if no hot days — handle that safely
        onset_val = doy_hot.min(dim="date").values
        try:
            onset_doy = float(onset_val) if np.isfinite(onset_val) else np.nan
        except TypeError:
            onset_doy = np.nan

        # Region name from shapefile
        zone_name = zones.iloc[rid][zone_col]

        rows.append({
            "year": int(year),
            "zone_index": rid,
            "zone_name": zone_name,
            "mean_tmax_MAMJ": mean_tmax,
            "p95_tmax_MAMJ": p95_tmax,
            f"hot_days_ge_{int(threshold)}C": hot_days,
            f"onset_doy_ge_{int(threshold)}C": onset_doy,
        })

    # free per-year stuff
    del ds_y, daily, daily_pre




Number of regions in mask: 32
Processing year: 1980
Processing year: 1981
Processing year: 1982
Processing year: 1983
Processing year: 1984
Processing year: 1985
Processing year: 1986
Processing year: 1987
Processing year: 1988
Processing year: 1989
Processing year: 1990
Processing year: 1991
Processing year: 1992
Processing year: 1993
Processing year: 1994
Processing year: 1995
Processing year: 1996
Processing year: 1997
Processing year: 1998
Processing year: 1999
Processing year: 2000
Processing year: 2001
Processing year: 2002
Processing year: 2003
Processing year: 2004
Processing year: 2005
Processing year: 2006
Processing year: 2007
Processing year: 2008
Processing year: 2009
Processing year: 2010
Processing year: 2011
Processing year: 2012
Processing year: 2013
Processing year: 2014
Processing year: 2015
Processing year: 2016
Processing year: 2017
Processing year: 2018
Processing year: 2019
Processing year: 2020
Processing year: 2021
Processing year: 2022
Processing year: 2023


In [9]:
# create a DataFrame from the results
df_metrics = pd.DataFrame(rows)
df_metrics.head()

,year,zone_index,zone_name,mean_tmax_MAMJ,p95_tmax_MAMJ,hot_days_ge_38C,onset_doy_ge_38C
0,1980,0,Andaman & Nicobar,29.485012,30.785149,0,NaN
1,1980,1,Andhra Pradesh,36.602806,41.371344,49,83.0
2,1980,2,Assam,28.479902,31.295429,0,NaN
3,1980,3,Chandigarh,34.445499,40.890350,42,110.0
4,1980,4,Dadra and Nagar Haveli,34.332851,38.512435,13,87.0


In [10]:
# save to CSV
out_csv = os.path.join(
    OUT_DIR,
    f"heat_metrics_MarJun_{int(threshold)}C.csv"
)
df_metrics.to_csv(out_csv, index=False)
out_csv


'/Users/niko2485/Library/CloudStorage/OneDrive-UCB-O365/Desktop/EarthScienceCourse/Final project/output/heat_metrics_MarJun_38C.csv'